# Обработка договоров хакатона

In [3]:
# импорт необходимых библеотек
import json
import sqlite3
import numpy as np
import pandas as pd
import aspose.words as aw

In [144]:
# считывание файла с видами договоров
with open('data/classes.json', 'r', encoding='utf8') as f:
    classes = json.load(f)

In [145]:
# вывод видов договоров
classes

{'5908cb5da047d6c9e6dfea6337fb3189.doc': 'Договоры для акселератора/Договоры поставки',
 '14711e4fc8e56f0c75856c8837ec04cb.doc': 'Договоры для акселератора/Договоры поставки',
 '7eb67b5aecf3f3190aab0a5f8ea32172.docx': 'Договоры для акселератора/Договоры поставки',
 'b40a9d048b199d5f4db62a6a2335f2a0.pdf': 'Договоры для акселератора/Договоры поставки',
 '84fec112d02288861e7af59f468131fb.docx': 'Договоры для акселератора/Договоры поставки',
 'f6377999f8a5aa9a09b03e428ac93153.doc': 'Договоры для акселератора/Договоры поставки',
 'a525f050cef10dee3a42468daec064ff.doc': 'Договоры для акселератора/Договоры поставки',
 'bec0aa38d1383172690a18d16b07f154.doc': 'Договоры для акселератора/Договоры поставки',
 '214d620d9c54bc83111277dd872d3cb2.pdf': 'Договоры для акселератора/Договоры поставки',
 'd143c89d002fcef3e2bd2efdb4966f55.doc': 'Договоры для акселератора/Договоры поставки',
 '2fd747f38e30ae7ce1c9d6e3b907ac5d.doc': 'Договоры для акселератора/Договоры поставки',
 '4c2c295e81f4a6c3e669e8f76c6c

In [6]:
# наименования классов
kind_names = np.unique(list(classes.values()))
kind_names

array(['Договоры для акселератора/Договоры аренды',
       'Договоры для акселератора/Договоры купли-продажи',
       'Договоры для акселератора/Договоры оказания услуг',
       'Договоры для акселератора/Договоры подряда',
       'Договоры для акселератора/Договоры поставки'], dtype='<U49')

In [7]:
# очистка наименований классов и подготовка к записи в базу данных
kind_names = [(name.split('/')[-1].replace('ы', '', 1),) for name in kind_names]
kind_names

[('Договор аренды',),
 ('Договор купли-продажи',),
 ('Договор оказания услуг',),
 ('Договор подряда',),
 ('Договор поставки',)]

In [4]:
# подключение к базе данных (в случае отсутствия база данных создается)
connection = sqlite3.connect('contracts.db')
# создание объекта подключения
cursor = connection.cursor()
print("База данных создана и успешно подключена к SQLite")

База данных создана и успешно подключена к SQLite


In [9]:
# наименование файлов договора
classes_keys = list(classes.keys())
classes_keys[:5]

['5908cb5da047d6c9e6dfea6337fb3189.doc',
 '14711e4fc8e56f0c75856c8837ec04cb.doc',
 '7eb67b5aecf3f3190aab0a5f8ea32172.docx',
 'b40a9d048b199d5f4db62a6a2335f2a0.pdf',
 '84fec112d02288861e7af59f468131fb.docx']

In [10]:
# количество файлов
len(classes_keys)

120

In [11]:
# функция записи наименования вида договора в таблицу contract_kinds
def InsertContractClassName(kind_name):
    
    #connection = sqlite3.connect('contracts.db')
    #cursor = connection.cursor()
    cursor.executemany("INSERT INTO contract_kinds (kind_name) VALUES(?)", kind_name)
    connection.commit()
    #connection.close()

In [12]:
# запись наименований в базу данных
InsertContractClassName(kind_names)

In [5]:
# проверка внесения записи
cursor.execute("SELECT * FROM contract_kinds;")
three_results = cursor.fetchall()
print(three_results)

[(1, 'Договор аренды'), (2, 'Договор купли-продажи'), (3, 'Договор оказания услуг'), (4, 'Договор подряда'), (5, 'Договор поставки')]


In [31]:
# подготовка id наименований вида договора и наименования вида договора к записи в файл в виде словаря
kind_id = [result[0] for result in three_results]
kind_name = [result[1] for result in three_results]

In [32]:
# преобразование в словарь
kind_names = dict(zip(kind_id, kind_name))
kind_names

{1: 'Договор аренды',
 2: 'Договор купли-продажи',
 3: 'Договор оказания услуг',
 4: 'Договор подряда',
 5: 'Договор поставки'}

In [33]:
# запись словаря в файл
with open('data/kind_names.json', 'w') as outfile:
    json.dump(kind_names, outfile)

In [34]:
# проверка записи в файл
with open('data/kind_names.json', 'r') as f:
    kind_names = json.load(f)

In [35]:
kind_names

{'1': 'Договор аренды',
 '2': 'Договор купли-продажи',
 '3': 'Договор оказания услуг',
 '4': 'Договор подряда',
 '5': 'Договор поставки'}

In [148]:
# извлечение наименований видов договоров
cursor.execute("SELECT kind_name FROM contract_kinds;")
three_results = cursor.fetchall()
print(three_results)

[('Договор аренды',), ('Договор купли-продажи',), ('Договор оказания услуг',), ('Договор подряда',), ('Договор поставки',)]


In [149]:
# очистка информации
[_[0] for _ in three_results]

['Договор аренды',
 'Договор купли-продажи',
 'Договор оказания услуг',
 'Договор подряда',
 'Договор поставки']

In [121]:
# функция извлечения kind_id из базы данных
def SelectKindId(kind_name):
    
    #connection = sqlite3.connect('contracts.db')
    #cursor = connection.cursor()
    cursor.execute("SELECT kind_id FROM contract_kinds WHERE kind_name = ?;", (kind_name,))
    kind_id = cursor.fetchone()[0]
    #connection.close()
    
    return kind_id

In [122]:
# функция определения текущей даты greate_date
def SelectDateNow():
    
    #connection = sqlite3.connect('contracts.db')
    #cursor = connection.cursor()
    cursor.execute("SELECT date('now');")
    greate_date = cursor.fetchone()[0]
    #connection.close()
    
    return  greate_date

In [123]:
# функция записи договора в базу данных
def InsertContract(kind_id, greate_date, contract_name, contract):
    
    #connection = sqlite3.connect('contracts.db')
    #cursor = connection.cursor()
    cursor.execute("INSERT INTO contracts \
              (kind_id, greate_date, contract_name, contract) \
              VALUES(?, ?, ?, ?)", \
              (kind_id, greate_date, contract_name, contract))
    connection.commit()
    #connection.close()

In [20]:
%%time
# внесение договоров в базу данных
for file in classes_keys:
    
    # имя файла договора
    contract_name = file
    # путь к файлу
    path = 'data/docs/' + file
    # загрузка файла договора
    data = aw.Document(path)
    # извлечение текста договора
    contract = data.get_text()
    # очистка от рекламного текста 
    contract = contract.replace('Evaluation Only. Created with Aspose.Words. Copyright 2003-2022 Aspose Pty Ltd.', '')
    contract = contract.replace('Created with an evaluation copy of Aspose.Words. To discover the full versions of our APIs please visit: https://products.aspose.com/words/', '')
    # наименование вида договора
    kind_name = classes[file].split('/')[-1].replace('ы', '', 1)
    # id наименования вида договора
    kind_id = SelectKindId(kind_name)
    # дата сохранения договора в базу данных
    greate_date = SelectDateNow()
    # запись договора в базу данных   
    InsertContract(kind_id, greate_date, contract_name, contract)

CPU times: total: 20.5 s
Wall time: 22.6 s


In [124]:
# преобразование данных в DaraFrame
cursor.execute("SELECT contract_id, kind_id, kind_name, greate_date, contract_name, contract \
                FROM contracts \
                INNER JOIN contract_kinds USING (kind_id);")
contracts = cursor.fetchall()
colnames = ['contract_id', 'kind_id', 'kind_name', 'greate_date', 'contract_name', 'contract']
contracts_df = pd.DataFrame(data=contracts, columns=colnames)
contracts_df.head()

,contract_id,kind_id,kind_name,greate_date,contract_name,contract
0,1,5,Договор поставки,2022-12-26,5908cb5da047d6c9e6dfea6337fb3189.doc,\r\t\tДОГОВОР \r\rг. Москва«__» ________ 2012...
1,2,5,Договор поставки,2022-12-26,14711e4fc8e56f0c75856c8837ec04cb.doc,"\r\r\rДоговор №______________\r\r\r Дата, мест..."
2,3,5,Договор поставки,2022-12-26,7eb67b5aecf3f3190aab0a5f8ea32172.docx,\r\r\t\t\r\rДОГОВОР ЗАКУПКИ № __________/\r\rг...
3,4,5,Договор поставки,2022-12-26,b40a9d048b199d5f4db62a6a2335f2a0.pdf,\rДОГОВОР ПОСТАВКИ № 1 \rг. Москва «11...
4,5,5,Договор поставки,2022-12-26,84fec112d02288861e7af59f468131fb.docx,\rДоговор № {НомерДокумента}\r{ДатаДокумента...


In [125]:
# размерность данных
contracts_df.shape

(120, 6)

In [126]:
# проверка наличия пропусков
contracts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   contract_id    120 non-null    int64 
 1   kind_id        120 non-null    int64 
 2   kind_name      120 non-null    object
 3   greate_date    120 non-null    object
 4   contract_name  120 non-null    object
 5   contract       120 non-null    object
dtypes: int64(2), object(4)
memory usage: 5.8+ KB


In [127]:
# сохранение в файл
contracts_df.to_csv('data/contracts_df.csv', index=False)

In [36]:
# закрытие соединения с базой данных
connection.close()